# Making pool of values for DOE - BO experiment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# Length = [28.639, 28.276, 31.121, 19.835, 33.654, 30.594, 29.529]#np.arange(12,51)
# Width = [9.008, 9.004, 7.358, 14.762, 6.926, 8.897, 8.737]#np.arange(6,26)
Length = [28.639, 31.121, 33.654, 30.594]#np.arange(12,51)
Width = [9.008, 7.358, 6.926, 8.897]#np.arange(6,26)
Thickness = 1.22 #nm average(?)
Unit_volume = 0.221445125 #nm^3
epsilon = 31947800  # (1/M*cm)ROUGH ESTIMATE from JPCC 2015, 119, 26768		 *for 4 ML NPLs with lateral size of ~300nm^2	
Cd_atoms_per_unit_volume = 4
Se_atoms_per_unit_volume = 0.875
Ag_acet_concentration = [0.006,0.0007] #M
Dilution_factor_for_OD = 100
Ag_percentage_doping = [2,5,10]  # % of doping
Optical_density_at_dilution_factor_100 = [0.15,0.3]  #Experimentally measured  
Aliquote = [0.25,0.5] #mL of NPL Aliquote for experiments from Ag_acet_concentration
Mass_Ag_acet = [3.5,7.8,14] #mg Ag_acet
Ag_Acet_mw = 166.91 #g/mol



In [ ]:
# Create nested loops to iterate through the possible values of each parameter
import pandas as pd 
from tqdm import tqdm
from itertools import product
data = { 'Dimension1(nm)': [], 'Dimension2(nm)': [],  'Dilution(%)': [], 'Nano_stock_vol(mL)': [], 
        'Diluted_vol(mL)': [], 'AgConc(M)': [], 'AgMass(mg)': [], 'MeOH_vol(mL)': [], 
        'H2O_vol(mL)': [], 'Doping(%)': [], 'AgSol(uL)': [],'Temperature(C)': [],'Time(min)':[]}
Time = [10, 60, 180] # min
Temperature = [0, 25, 50] # C
areas = []
#get combinations of L and W without considering order
# L_w_combs = list(product(Length, Width))
L_w_combs = [(L,W) for (L, W) in zip(Length, Width)]
print(L_w_combs)
depurated_L_w_combs = []
areas = []
for L, W in L_w_combs:
    if L*W not in areas:
        areas.append(L*W)
        depurated_L_w_combs.append((L,W))

total_iterations = len(depurated_L_w_combs) * len([Dilution_factor_for_OD]) * len(Optical_density_at_dilution_factor_100) * len(Aliquote) * len(Ag_acet_concentration) * len(Mass_Ag_acet) * len(Ag_percentage_doping)
with tqdm(total=total_iterations) as pbar:
    for DF in [Dilution_factor_for_OD]:
        for OD in Optical_density_at_dilution_factor_100:
            for NSV in Aliquote:
                for AAC in Ag_acet_concentration:
                    for AM in Mass_Ag_acet:
                        for DP in Ag_percentage_doping:
                            for L, W in depurated_L_w_combs:
                                    # Calculations based on spreadsheet
                                    pbar.update(1)

                                    Unique_unit_areas = L * W
                                    Unique_unit_volumes = Unique_unit_areas * Thickness  # nm^3
                                    Unique_number_qd = Unique_unit_volumes / Unit_volume  # NUMBER OF QD per unit

                                    Concentration_at_dilution = OD / epsilon  # [NPL] M at dilution
                                    Concentration = Concentration_at_dilution * DF  # [NPL] M
                                    Cd_concentration = Concentration * Unique_number_qd * Cd_atoms_per_unit_volume # [Cd] M= [NPL] M * qd_number per NPL * Cd_atoms_per_unit_volume

                                    # Solution of ag_acet to get Ag_mol of 0.006M or 0.0007M from Ag_acet mass
                                    Ag_total_sol_volume = (AM / Ag_Acet_mw) / AAC  # mL
                                    Me_OH_vol = Ag_total_sol_volume * 5 / 7  # mL MeOH
                                    Water_vol = Ag_total_sol_volume * 2 / 7  # mL H2O
                                    Cd_mols = Cd_concentration * NSV / 1000
                                    Ag_mols_needed = Cd_mols * DP / 100  # mol of Ag needed
                                    Vol_Ag_acet_needed = Ag_mols_needed / AAC * 1000 * 1000 #micro liters
                                    # Append values to lists
                                    for T in Temperature:
                                        for t in Time:
                                            data['Dimension1(nm)'].append(L)
                                            data['Dimension2(nm)'].append(W)
                                            data['Dilution(%)'].append(OD)    #wrong name, it should be OD w dilution
                                            data['Nano_stock_vol(mL)'].append(NSV)
                                            data['Diluted_vol(mL)'].append(NSV*7)
                                            data['AgConc(M)'].append(AAC)
                                            data['AgMass(mg)'].append(AM)
                                            data['MeOH_vol(mL)'].append(Me_OH_vol)
                                            data['H2O_vol(mL)'].append(Water_vol)
                                            data['Doping(%)'].append(DP)
                                            data['AgSol(uL)'].append(Vol_Ag_acet_needed)
                                            data['Time(min)'].append(t)
                                            data['Temperature(C)'].append(T)

data = pd.DataFrame(data)
display(data)

In [ ]:
print(len(data))

In [4]:
data.to_csv('Data/pool_processed.csv', index=False)